In [1]:
import torch
from core.model import GPT, GPTConfig
from core.tokenizer import Tokenizer, AudioTokenizer
from huggingface_hub import hf_hub_download
import numpy as np

In [2]:
model_config = GPTConfig.from_pretrained('EleutherAI/pythia-410m')
model = GPT(model_config)
tokenizer = Tokenizer()
audio_tokenizer = AudioTokenizer()

number of parameters: 409.55M
Loading Audio Encoder


Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration.


Loaded Audio Encoder, size: 1029


In [3]:
# Step 1, load the model

model_name = 'alexedw/audio-clean-100-run-2'
model_checkpoint = '750'

state_dict = torch.load(hf_hub_download(model_name, "model_state.pt", revision=model_checkpoint), map_location='cpu')
new_state_dict = {key.replace("_orig_mod.", ""): value for key, value in state_dict.items()}

model.load_state_dict(new_state_dict)

<All keys matched successfully>

In [4]:
# prepare tokens
input_text = 'my name is alexander and I am so great'
tokenized_text = tokenizer.encode(input_text)

full_text_tokens = np.concatenate([np.array([tokenizer.start_text_id]), tokenized_text, np.array([tokenizer.end_text_id])])
full_audio_tokens = np.concatenate([np.array([audio_tokenizer.start_text_id]), np.array([audio_tokenizer.text_id] * len(tokenized_text)), np.array([audio_tokenizer.end_text_id])])

full_text_tokens = torch.tensor(full_text_tokens).unsqueeze(0)
full_audio_tokens_1 = torch.tensor(full_audio_tokens).unsqueeze(0)
full_audio_tokens_2 = torch.tensor(full_audio_tokens).unsqueeze(0)

In [5]:
# run generation
output_text, output_audio_1, output_audio_2 = model.generate(full_text_tokens, full_audio_tokens_1, full_audio_tokens_2, 500)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


 10%|▉         | 48/500 [00:20<03:40,  2.05it/s]